<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/220107_ji_catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 
import tensorflow as tf
import keras
from numpy import array
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib as mpl
from IPython.core.display import display, HTML
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, f1_score
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, LSTM, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
import urllib.request
import json
import datetime
import os
from datetime import datetime
from tqdm import tqdm
import warnings
from glob import glob
from catboost import CatBoostClassifier

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
warnings.filterwarnings(action='ignore') 
train1 = pd.read_csv('/content/gdrive/MyDrive/data/know/train/KNOW_2017.csv', encoding='utf-8')
train2 = pd.read_csv('/content/gdrive/MyDrive/data/know/train/KNOW_2018.csv', encoding='utf-8')
train3 = pd.read_csv('/content/gdrive/MyDrive/data/know/train/KNOW_2019.csv', encoding='utf-8')
train4 = pd.read_csv('/content/gdrive/MyDrive/data/know/train/KNOW_2020.csv', encoding='utf-8')

test1 = pd.read_csv('/content/gdrive/MyDrive/data/know/test/KNOW_2017_test.csv', encoding='utf-8')
test2 = pd.read_csv('/content/gdrive/MyDrive/data/know/test/KNOW_2018_test.csv', encoding='utf-8')
test3 = pd.read_csv('/content/gdrive/MyDrive/data/know/test/KNOW_2019_test.csv', encoding='utf-8')
test4 = pd.read_csv('/content/gdrive/MyDrive/data/know/test/KNOW_2020_test.csv', encoding='utf-8')

submission = pd.read_csv('/content/gdrive/MyDrive/data/know/sample_submission.csv', encoding='utf-8')

train2 = train2[train2['idx'] != 18048]
train3 = train3[train3['idx'] != 19871]

In [ ]:
#bq4_1a, bq4_1b, bq4_1c, bq5_2, bq19_1, bq30, bq31, bq32, bq33, bq34, bq38_1
# 'bq5_2', 'bq19_1'은 삭제 'bq31'은 피처 123

cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq30', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1']


In [ ]:
train1['bq31']

0                                   없다
1                   매타기, 드라이버, 가위, CAD
2                건축설계표, 오토캐드, 파이프, 보일러
3                          토탈스테이션, SZP
4                         캐드, 엑셀, 건축도면
                     ...              
9481             한글, 편집프로그램, 포토샵, 일러스트
9482    AUTO CAD , CATIA, 전문 특수정비도구(툴)
9483                            볼트, 리벳
9484                               컴퓨터
9485                          차량관리 시스템
Name: bq31, Length: 9486, dtype: object

In [ ]:
def tool_to_feature(text:str):
  result = []
  for i in text.split(','):
    result.append(i)
  return result

train1['bq31'] = train1['bq31'].map(tool_to_feature)

def list_truncate(tool:list):
  if len(tool) > 3:
    return tool[:3]
  elif len(tool) == 3:
    return tool
  elif len(tool) == 2:
    return tool+['0']
  elif len(tool) == 1:
    return tool+['0', '0']
  elif len(tool) == 0:
    return ['0', '0', '0']

train1['bq31'] = train1['bq31'].map(list_truncate)

def list_to_feature1(tool:list):
  return tool[0]

def list_to_feature2(tool:list):
  return tool[1]

def list_to_feature3(tool:list):
  return tool[2]


feature1 = train1['bq31'].map(list_to_feature1)
feature2 = train1['bq31'].map(list_to_feature2)
feature3 = train1['bq31'].map(list_to_feature3)

train1['feature1'] = feature1
train1['feature2'] = feature2
train1['feature3'] = feature3

train1[['feature1', 'feature2', 'feature3']]

,feature1,feature2,feature3
0,없다,0,0
1,매타기,드라이버,가위
2,건축설계표,오토캐드,파이프
3,토탈스테이션,SZP,0
4,캐드,엑셀,건축도면
...,...,...,...
9481,한글,편집프로그램,포토샵
9482,AUTO CAD,CATIA,전문 특수정비도구(툴)
9483,볼트,리벳,0
9484,컴퓨터,0,0


In [ ]:
train1 = train1.iloc[:,1:]
train1

,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,aq5_2,aq6_1,aq6_2,aq7_1,aq7_2,aq8_1,aq8_2,aq9_1,aq9_2,aq10_1,aq10_2,aq11_1,aq11_2,aq12_1,aq12_2,aq13_1,aq13_2,aq14_1,aq14_2,aq15_1,aq15_2,aq16_1,aq16_2,aq17_1,aq17_2,aq18_1,aq18_2,aq19_1,aq19_2,aq20_1,aq20_2,...,bq18_7,bq19,bq19_1,bq20,bq21,bq22,bq23,bq24_1,bq24_2,bq24_3,bq24_4,bq24_5,bq24_6,bq24_7,bq24_8,bq25,bq26,bq27,bq28,bq29,bq30,bq31,bq32,bq33,bq34,bq35,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3,knowcode,feature1,feature2,feature3
0,3,3,3,3,3,3,4,4,3,4,3,3,2,2,2,2,2,2,3,3,3,3,4,4,3,3,3,2,3,3,5,6,5,6,4,5,2,2,5,6,...,3,2,생산설비의 자동화로,2,3,5,10.0,1,1,1,2,2,2,2,2,3,3,4,3,2,없다,"[없다, 0, 0]",없다,건설현장 노무직,없다,4,1,52,2,실업,1,1,1,4000,,2200,825101,없다,0,0
1,4,5,4,5,3,4,3,4,3,4,1,,1,,1,,1,,2,3,3,4,3,4,4,5,2,2,3,4,3,4,3,4,3,4,1,,2,3,...,4,2,건설 수주가 없어서,3,3,6,50.0,2,2,2,2,2,2,2,2,4,3,4,3,2,없다,"[매타기, 드라이버, 가위]",,,,2,1,38,4,건축공학,1,1,1,,,2400,140204,매타기,드라이버,가위
2,3,4,3,4,3,4,5,6,4,5,1,,1,,3,4,3,4,3,4,1,,3,4,3,4,1,,3,4,3,4,3,4,3,4,3,4,3,4,...,4,2,신축 건설경기가 좋지 않아서,3,2,5,40.0,2,2,2,2,2,2,2,2,4,4,4,2,2,없다,"[건축설계표, 오토캐드, 파이프]",,,,2,1,50,4,건축공학,1,1,1,4000,,2400,140204,건축설계표,오토캐드,파이프
3,3,3,3,3,3,5,4,5,4,6,3,5,3,4,4,5,5,5,4,5,3,4,4,5,3,5,3,4,4,4,3,4,2,2,4,4,4,2,4,3,...,3,1,"업무량 감소, 자동화로 인해서",4,2,6,30.0,2,1,1,2,1,1,1,1,3,4,4,4,4,지적기사,"[토탈스테이션, SZP, 0]",무,지적공무원,,4,1,42,4,환경학과,1,1,1,7000,,3500,140601,토탈스테이션,SZP,0
4,4,5,3,4,3,4,4,5,3,4,1,,1,,1,,1,,3,4,2,2,3,4,3,4,1,,3,4,3,4,3,4,3,4,1,,2,3,...,4,2,부동산 거품 빠지면서 건축 경기도 덩달아 불황,3,2,3,40.0,2,2,2,2,2,2,2,2,4,3,4,3,3,없다,"[캐드, 엑셀, 건축도면]",,,,2,1,51,4,건축공학,1,1,1,4000,,2500,140204,캐드,엑셀,건축도면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,3,5,2,4,3,3,2,2,2,3,2,3,3,3,2,3,2,3,3,4,4,5,4,5,3,4,2,3,3,4,2,2,2,2,2,2,3,5,1,,...,3,3,출판업무 특성상 기획업무는 변함없음,2,3,1,10.0,2,1,2,2,2,2,2,2,4,4,4,4,3,없다,"[한글, 편집프로그램, 포토샵]",없다,마케팅업무,없다,2,1,50,4,산업디자인,1,1,1,5200,,1800,411301,한글,편집프로그램,포토샵
9482,5,5,5,5,5,5,3,4,4,5,5,5,4,4,5,5,4,5,4,5,4,5,5,5,4,5,4,5,5,4,2,2,3,4,4,5,5,5,3,4,...,4,4,항공우주분야는 국가 경쟁력을 위해 육성해야 되는 분야이므로,3,3,5,20.0,2,2,1,2,2,2,2,2,4,4,3,4,3,없다,"[AUTO CAD , CATIA, 전문 특수정비도구(툴)]",없다,없다,없다,2,1,37,4,우주항공공학,1,1,1,4000,,3000,151105,AUTO CAD,CATIA,전문 특수정비도구(툴)
9483,3,3,4,6,3,3,4,5,4,5,3,4,3,4,1,,2,2,3,3,3,4,3,4,3,4,3,4,3,3,5,7,5,7,4,6,1,,4,5,...,3,3,건설경기는 꾸준해서,2,3,1,0.0,2,2,2,2,2,2,2,2,3,3,4,3,1,건설 노가대,"[볼트, 리벳, 0]",없다,없다,없다,2,1,32,2,기계,1,4,2,2700,,1500,701101,볼트,리벳,0
9484,3,5,3,5,4,5,3,4,3,5,4,6,4,5,4,5,4,6,4,6,3,4,3,5,3,4,2,3,4,6,3,3,3,4,3,3,4,5,4,4,...,3,3,공무원의 인원은 크게 변동이 없어서,2,2,5,20.0,2,1,1,2,1,2,1,1,3,4,4,3,2,없다,"[컴퓨터, 0, 0]",없다,모름,,2,2,40,4,문헌정보학,1,1,1,6800,,2500,25402,컴퓨터,0,0


In [ ]:
# 필요없는 칼럼 삭제
train1 = train1.drop(columns=['bq31','bq5_2','bq19_1'])
train1

,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,aq5_2,aq6_1,aq6_2,aq7_1,aq7_2,aq8_1,aq8_2,aq9_1,aq9_2,aq10_1,aq10_2,aq11_1,aq11_2,aq12_1,aq12_2,aq13_1,aq13_2,aq14_1,aq14_2,aq15_1,aq15_2,aq16_1,aq16_2,aq17_1,aq17_2,aq18_1,aq18_2,aq19_1,aq19_2,aq20_1,aq20_2,...,bq18_5,bq18_6,bq18_7,bq19,bq20,bq21,bq22,bq23,bq24_1,bq24_2,bq24_3,bq24_4,bq24_5,bq24_6,bq24_7,bq24_8,bq25,bq26,bq27,bq28,bq29,bq30,bq32,bq33,bq34,bq35,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3,knowcode,feature1,feature2,feature3
0,3,3,3,3,3,3,4,4,3,4,3,3,2,2,2,2,2,2,3,3,3,3,4,4,3,3,3,2,3,3,5,6,5,6,4,5,2,2,5,6,...,2,3,3,2,2,3,5,10.0,1,1,1,2,2,2,2,2,3,3,4,3,2,없다,없다,건설현장 노무직,없다,4,1,52,2,실업,1,1,1,4000,,2200,825101,없다,0,0
1,4,5,4,5,3,4,3,4,3,4,1,,1,,1,,1,,2,3,3,4,3,4,4,5,2,2,3,4,3,4,3,4,3,4,1,,2,3,...,2,2,4,2,3,3,6,50.0,2,2,2,2,2,2,2,2,4,3,4,3,2,없다,,,,2,1,38,4,건축공학,1,1,1,,,2400,140204,매타기,드라이버,가위
2,3,4,3,4,3,4,5,6,4,5,1,,1,,3,4,3,4,3,4,1,,3,4,3,4,1,,3,4,3,4,3,4,3,4,3,4,3,4,...,2,2,4,2,3,2,5,40.0,2,2,2,2,2,2,2,2,4,4,4,2,2,없다,,,,2,1,50,4,건축공학,1,1,1,4000,,2400,140204,건축설계표,오토캐드,파이프
3,3,3,3,3,3,5,4,5,4,6,3,5,3,4,4,5,5,5,4,5,3,4,4,5,3,5,3,4,4,4,3,4,2,2,4,4,4,2,4,3,...,2,2,3,1,4,2,6,30.0,2,1,1,2,1,1,1,1,3,4,4,4,4,지적기사,무,지적공무원,,4,1,42,4,환경학과,1,1,1,7000,,3500,140601,토탈스테이션,SZP,0
4,4,5,3,4,3,4,4,5,3,4,1,,1,,1,,1,,3,4,2,2,3,4,3,4,1,,3,4,3,4,3,4,3,4,1,,2,3,...,2,2,4,2,3,2,3,40.0,2,2,2,2,2,2,2,2,4,3,4,3,3,없다,,,,2,1,51,4,건축공학,1,1,1,4000,,2500,140204,캐드,엑셀,건축도면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,3,5,2,4,3,3,2,2,2,3,2,3,3,3,2,3,2,3,3,4,4,5,4,5,3,4,2,3,3,4,2,2,2,2,2,2,3,5,1,,...,3,2,3,3,2,3,1,10.0,2,1,2,2,2,2,2,2,4,4,4,4,3,없다,없다,마케팅업무,없다,2,1,50,4,산업디자인,1,1,1,5200,,1800,411301,한글,편집프로그램,포토샵
9482,5,5,5,5,5,5,3,4,4,5,5,5,4,4,5,5,4,5,4,5,4,5,5,5,4,5,4,5,5,4,2,2,3,4,4,5,5,5,3,4,...,1,2,4,4,3,3,5,20.0,2,2,1,2,2,2,2,2,4,4,3,4,3,없다,없다,없다,없다,2,1,37,4,우주항공공학,1,1,1,4000,,3000,151105,AUTO CAD,CATIA,전문 특수정비도구(툴)
9483,3,3,4,6,3,3,4,5,4,5,3,4,3,4,1,,2,2,3,3,3,4,3,4,3,4,3,4,3,3,5,7,5,7,4,6,1,,4,5,...,4,3,3,3,2,3,1,0.0,2,2,2,2,2,2,2,2,3,3,4,3,1,건설 노가대,없다,없다,없다,2,1,32,2,기계,1,4,2,2700,,1500,701101,볼트,리벳,0
9484,3,5,3,5,4,5,3,4,3,5,4,6,4,5,4,5,4,6,4,6,3,4,3,5,3,4,2,3,4,6,3,3,3,4,3,3,4,5,4,4,...,3,2,3,3,2,2,5,20.0,2,1,1,2,1,2,1,1,3,4,4,3,2,없다,없다,모름,,2,2,40,4,문헌정보학,1,1,1,6800,,2500,25402,컴퓨터,0,0


In [ ]:
X_2017 = train1.drop(columns=['knowcode'])
y_2017 = train1['knowcode']


X_train_2017,X_test_2017,y_train_2017,y_test_2017 = train_test_split(X_2017,y_2017,test_size=0.2,random_state=10)

In [ ]:
cat_features = list()
for i in X_train_2017.columns:
    cat_features.append(i)
print(cat_features)

del_features = ['bq23', 'bq37', 'bq41_1', 'bq41_2', 'bq41_3']
for i in del_features:
  cat_features.remove(i)
print(cat_features)

['aq1_1', 'aq1_2', 'aq2_1', 'aq2_2', 'aq3_1', 'aq3_2', 'aq4_1', 'aq4_2', 'aq5_1', 'aq5_2', 'aq6_1', 'aq6_2', 'aq7_1', 'aq7_2', 'aq8_1', 'aq8_2', 'aq9_1', 'aq9_2', 'aq10_1', 'aq10_2', 'aq11_1', 'aq11_2', 'aq12_1', 'aq12_2', 'aq13_1', 'aq13_2', 'aq14_1', 'aq14_2', 'aq15_1', 'aq15_2', 'aq16_1', 'aq16_2', 'aq17_1', 'aq17_2', 'aq18_1', 'aq18_2', 'aq19_1', 'aq19_2', 'aq20_1', 'aq20_2', 'aq21_1', 'aq21_2', 'aq22_1', 'aq22_2', 'aq23_1', 'aq23_2', 'aq24_1', 'aq24_2', 'aq25_1', 'aq25_2', 'aq26_1', 'aq26_2', 'aq27_1', 'aq27_2', 'aq28_1', 'aq28_2', 'aq29_1', 'aq29_2', 'aq30_1', 'aq30_2', 'aq31_1', 'aq31_2', 'aq32_1', 'aq32_2', 'aq33_1', 'aq33_2', 'aq34_1', 'aq34_2', 'aq35_1', 'aq35_2', 'aq36_1', 'aq36_2', 'aq37_1', 'aq37_2', 'aq38_1', 'aq38_2', 'aq39_1', 'aq39_2', 'aq40_1', 'aq40_2', 'aq41_1', 'aq41_2', 'bq1', 'bq2', 'bq3', 'bq4', 'bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5', 'bq5_1', 'bq6', 'bq7', 'bq8_1', 'bq8_2', 'bq8_3', 'bq9', 'bq10', 'bq11', 'bq12_1', 'bq12_2', 'bq12_3', 'bq12_4', 'bq12_5', 'bq13', '

In [ ]:
text_features =  ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq30', 'bq32', 'bq33', 'bq34', 'bq38_1']

In [ ]:
model = CatBoostClassifier(task_type='GPU',iterations=500, depth=5)

model.fit(X_train_2017, y_train_2017, cat_features, verbose=True)

pred = model.predict(X_test_2017)

In [ ]:
pred = pd.DataFrame(pred)
pred[0]

0        531601
1       9999999
2        307501
3         24402
4       9999999
         ...   
1893    9999999
1894     214301
1895     821101
1896     831301
1897    9999999
Name: 0, Length: 1898, dtype: int64

In [ ]:
y_test_2017

9148    531601
131     872101
1216    307501
655      24404
5143    811301
         ...  
3378     29201
7904    214301
4142    817201
4394    842301
364     885201
Name: knowcode, Length: 1898, dtype: int64

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test_2017.values, pred[0].values)

0.3519494204425711